In [2]:
import awkward as ak
import uproot
import numpy as np
from scipy.optimize import curve_fit


import sys
import datetime
import numpy as np
import pandas as pd
import csv
import os
import subprocess
import ROOT as rt
import itertools


REPO_BASE_PATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
plot_path = f"{REPO_BASE_PATH}/plots/ADR/time_resolution_202503/"
os.makedirs(plot_path, exist_ok=True)

sys.path.append(REPO_BASE_PATH + "/lib")
from histo_utilities import *

import tdrstyle
a = tdrstyle.setTDRStyle()

Welcome to JupyROOT 6.30/02


In [17]:
files_raw = {}
tree_raw = {}

# Open the input ROOT file
voltages = np.round([0.160],3)
biases = [4,1,2] #slot number of voltage source
pixels = [1,2,3] # pixel number in SNSPD
scope_ch = [2,3,4] # channel on scope


inputDirBase = "root://cmseos.fnal.gov///eos/uscms/store/user/christiw/SNSPD_data/ADR_time_resolution_202503/"
# inputDirBase = "/eos/uscms/store/user/christiw/SNSPD_data/ADR_time_resolution_202503/"
tempToDir = {
0.2: "20250311_0p2K",
# 0.5: "20250310_0p5K",
# 0.8: "20250309_0p8K",
# 1:"20250310_1p0K",

 }

# load raw files
for k, path in tempToDir.items():
    inputDir = f"{inputDirBase}/{path}/raw/"
    for ch_i, bias_ch in enumerate(biases): # loop over pixels
        name = f"B{bias_ch}_P{pixels[ch_i]}"
        for j, v in enumerate(voltages): # loop over voltages  
            bv_string = str(voltages[j]).replace(".","p")
            files_raw[f'{k}_{name}_{bv_string}'] = inputDir+f'output_run{name}_BV{bv_string}.root'
for k, v in files_raw.items():
    tree_raw[k] = uproot.open(v)["pulse"]
print(tree_raw.keys())

dict_keys(['0.2_B4_P1_0p16', '0.2_B1_P2_0p16', '0.2_B2_P3_0p16'])


# Plot waveforms

In [20]:
T_raw = tree_raw['0.2_B4_P1_0p16'].arrays(entry_stop=100)
T = tree['0.2_B4_P1_0p16'].arrays(entry_stop=100)

os.makedirs(f"{plot_path}/waveforms", exist_ok=True)

for i_evt in range(100):
    if not i_evt == 20:continue
    cond = T_raw['i_evt'] == i_evt
    ch = np.array(T_raw['channel'][cond])
    t = np.array(T_raw['time'][cond])
    slope = T['slope'][i_evt]
    intercept = T['intercept'][i_evt]
    t_start = T['t_10'][i_evt]
    t_stop = T['t_90'][i_evt]
    c = rt.TCanvas(f"c{i_evt}",f"c_{i_evt}",200,10,700,500);
    h = {}
    f = {}


    baseline = T['baseline'][i_evt]
    baseline_rms = T['baseline_rms'][i_evt]
    for i in [0,1]:# for laser and SNSPD
        waveform_baseline_subtracted = np.abs(ch[0,i,:] - baseline[i])
        min_val = np.min(waveform_baseline_subtracted)
        max_val = np.max(waveform_baseline_subtracted)
        norm_waveform = (waveform_baseline_subtracted - min_val) / (max_val - min_val)

        ## plot normalized waveforms
        h[i] = create_TGraph(t[0,i,:]*1e9, norm_waveform, axis_title = ['time [ns]','a.u.'])
        
        ## plot baseline subtracted waveforms
#         h[i] = create_TGraph(t[0,i,:]*1e9, waveform_baseline_subtracted, axis_title = ['time [ns]','a.u.'])
        
        
        ## plot raw waveforms
        #h[i] = create_TGraph(t[0,i,:]*1e9, ch[0,i,:], axis_title = ['time [ns]','Voltage [V]'])

        f[i] = rt.TF1(f"f{i}", f"{slope[i]}*x + {intercept[i]}", t_start[i], t_stop[i])

    for i, k in enumerate(h.keys()):
        h[k].SetMarkerColor(std_color_list[i])        
        h[k].Draw("ap" if i == 0 else 'p')
    
        f[k].SetLineWidth(2)
        f[k].SetLineColor(std_color_list[i+2])
        f[k].Draw('same')



    c.Draw()
    c.SaveAs(f'{plot_path}/waveforms/waveform_{i_evt}.png')
    c.SaveAs(f'{plot_path}/waveforms/waveform_{i_evt}.pdf')


1.2876932e-05 1.1644518
3.8574217e-07 0.113048635


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c20
Info in <TCanvas::Print>: png file /uscms_data/d3/christiw/SNSPD/SNSPD_timing_KeysightScope/plots/ADR/time_resolution_202503//waveforms/waveform_20.png has been created
Info in <TCanvas::Print>: pdf file /uscms_data/d3/christiw/SNSPD/SNSPD_timing_KeysightScope/plots/ADR/time_resolution_202503//waveforms/waveform_20.pdf has been created
